### An Implementation Of Batch Gradient Descent With $L_2$ Regularization And Early Stopping For Softmax Regression Without Using Scikit-learn

In [1]:
import numpy as np
np.random.seed(2042)

from sklearn import datasets
iris = datasets.load_iris()

In [2]:
def my_test_split(X, y, test_ratio = 0.2, validation_ratio = 0.2):
    total_size = len(X)
    test_size = int(total_size * test_ratio)
    valid_size = int(total_size * validation_ratio)
    train_size = total_size - test_size - valid_size
    
    permutation_indices = np.random.permutation(total_size)
    
    X_train = X[permutation_indices[:train_size]]
    y_train = y[permutation_indices[:train_size]]
    X_valid = X[permutation_indices[train_size:-test_size]]
    y_valid = y[permutation_indices[train_size:-test_size]]
    X_test = X[permutation_indices[-test_size:]]
    y_test = y[permutation_indices[-test_size:]]
    
    return X_train, y_train, X_valid, y_valid, X_test, y_test

In [3]:
def encode_output(y_output, possible_outputs):
    m = len(y_output)
    encoding = np.zeros((m, possible_outputs))
    encoding[np.arange(m), y_output] = 1
    return encoding

In [4]:
X = iris["data"][:, (2,3)]
y = iris["target"]

X_with_bias = np.c_[np.ones([len(X), 1]), X]

X_train, y_train, X_valid, y_valid, X_test, y_test = my_test_split(X_with_bias, y)

y_train_one_hot = encode_output(y_train, len(np.unique(y)))
y_valid_one_hot = encode_output(y_valid, len(np.unique(y)))
y_test_one_hot = encode_output(y_test, len(np.unique(y)))

In [5]:
def softmax(logits):
    exps = np.exp(logits)
    exps_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exps_sums

$J(\mathbf{\Theta}) =
- \dfrac{1}{m}\sum\limits_{i=1}^{m}\sum\limits_{k=1}^{K}{y_k^{(i)}\log\left(\hat{p}_k^{(i)}\right)}$

$\nabla_{\mathbf{\theta}^{(k)}} \, J(\mathbf{\Theta}) = \dfrac{1}{m} \sum\limits_{i=1}^{m}{ \left ( \hat{p}^{(i)}_k - y_k^{(i)} \right ) \mathbf{x}^{(i)}}$

In [6]:
n_inputs = X_train.shape[1]
n_outputs = len(np.unique(y))
m = len(X_train)
epsilon = 1e-7
iterations = 5001
eta = 0.01

theta = np.random.rand(n_inputs, n_outputs)
for i in range(iterations):
    logits = X_train.dot(theta)
    y_proba = softmax(logits)
    loss = -np.mean(np.sum(np.log(y_proba + epsilon) * y_train_one_hot, axis = 1))
    
    if i % 500 == 0:
        print(f"iteration: {i}, loss: {loss}")
    
    error = y_proba - y_train_one_hot

    gradients = 1/m * (X_train.T.dot(error))
    theta = theta - eta * gradients

iteration: 0, loss: 2.1041298095182546
iteration: 500, loss: 0.7975963791730118
iteration: 1000, loss: 0.6623252726598646
iteration: 1500, loss: 0.5815322702225296
iteration: 2000, loss: 0.5285580975272502
iteration: 2500, loss: 0.4907949263916581
iteration: 3000, loss: 0.46208653508183384
iteration: 3500, loss: 0.4391923433913653
iteration: 4000, loss: 0.4202717004630981
iteration: 4500, loss: 0.40420680212026194
iteration: 5000, loss: 0.39027932970205237


In [7]:
theta

array([[ 3.52684124, -0.18458978, -2.48941342],
       [ 0.09459761,  1.08547944,  0.92677895],
       [-1.37121463, -0.01431783,  1.96615452]])

In [8]:
logits = X_valid.dot(theta)
y_proba = softmax(logits)
y_predict = np.argmax(y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.9666666666666667

### With Regularization

In [9]:
n_inputs = X_train.shape[1]
n_outputs = len(np.unique(y))
m = len(X_train)
epsilon = 1e-7
iterations = 5001
eta = 0.1
alpha = 0.1

r_theta = np.random.rand(n_inputs, n_outputs)
for i in range(iterations):
    logits = X_train.dot(r_theta)
    y_proba = softmax(logits)
    entropy_loss = -np.mean(np.sum(np.log(y_proba + epsilon) * y_train_one_hot, axis = 1))
    l2_loss = 0.5 * np.sum(np.square(theta[1:]))
    loss = entropy_loss + l2_loss
    
    if i % 500 == 0:
        print(f"iteration: {i}, loss: {loss}")
    
    error = y_proba - y_train_one_hot
    gradients = 1/m * (X_train.T.dot(error)) + np.r_[np.zeros([1, n_outputs]), alpha * r_theta[1:]]
    r_theta = r_theta - eta * gradients

iteration: 0, loss: 4.7895118170337865
iteration: 500, loss: 4.354613634138111
iteration: 1000, loss: 4.303759867080397
iteration: 1500, loss: 4.281378596617913
iteration: 2000, loss: 4.269049337541218
iteration: 2500, loss: 4.261617688966042
iteration: 3000, loss: 4.256931573007697
iteration: 3500, loss: 4.2539000488526195
iteration: 4000, loss: 4.2519078975774125
iteration: 4500, loss: 4.250585567880129
iteration: 5000, loss: 4.24970206452791


In [10]:
logits = X_valid.dot(r_theta)
y_proba = softmax(logits)
y_predict = np.argmax(y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

1.0

### With Early Stopping

$J(\mathbf{\Theta}) =
- \dfrac{1}{m}\sum\limits_{i=1}^{m}\sum\limits_{k=1}^{K}{y_k^{(i)}\log\left(\hat{p}_k^{(i)}\right)}$

$\nabla_{\mathbf{\theta}^{(k)}} \, J(\mathbf{\Theta}) = \dfrac{1}{m} \sum\limits_{i=1}^{m}{ \left ( \hat{p}^{(i)}_k - y_k^{(i)} \right ) \mathbf{x}^{(i)}}$

In [48]:
n_iterations = 5001
n_inputs = X_train.shape[1]
n_outputs = len(np.unique(y))
e_theta = np.random.rand(n_inputs, n_outputs)
m = X_train.shape[0]
eta = 0.1
alpha = 0.1
best_loss = np.infty

for i in range(n_iterations):
    logits = X_train.dot(e_theta)
    proba = softmax(logits)
    
    label_proba = np.sum(proba * y_train_one_hot, axis=1)
    adjusted_label_proba = label_proba + epsilon
    log_label_proba = np.log(adjusted_label_proba)
    entropy_loss = -np.mean(log_label_proba)
    l2_loss = np.sum(np.square(e_theta[1:])) / 2
    loss = entropy_loss + alpha * l2_loss
    # loss = np.sum(-np.log(np.sum(proba * y_train_one_hot, axis=1) + epsilon))
    
    gradient_matrix = X_train.T.dot(proba - y_train_one_hot) / m + alpha * np.r_[np.zeros([1, n_outputs]), e_theta[1:]]
    e_theta -= eta * gradient_matrix
    logits = X_train.dot(e_theta)
    proba = softmax(logits)
    label_proba = np.sum(proba * y_train_one_hot, axis=1)
    adjusted_label_proba = label_proba + epsilon
    log_label_proba = np.log(adjusted_label_proba)
    entropy_loss = -np.mean(log_label_proba)
    l2_loss = np.sum(np.square(e_theta[1:])) / 2
    loss = entropy_loss + alpha * l2_loss
    
    if i % 500 == 0:
        print(f"iteration {i}, loss: {loss}")
    
    if loss < best_loss:
        best_loss = loss
    else:
        print(i-1, best_loss, "best loss")
        print(i, loss, "early stopping")
        break

iteration 0, loss: 1.0824958946408876
iteration 500, loss: 0.5283056103299459
iteration 1000, loss: 0.502075721417513
iteration 1500, loss: 0.4941190510171293
iteration 2000, loss: 0.4910668305306171
iteration 2500, loss: 0.48980162574234537
iteration 3000, loss: 0.489256432825036
iteration 3500, loss: 0.4890161892988417
iteration 4000, loss: 0.48890885997582456
iteration 4500, loss: 0.48886048947306704
iteration 5000, loss: 0.48883856594338904


In [49]:
logits = X_valid.dot(e_theta)
proba = softmax(logits)
y_valid_predict = np.argmax(proba, axis = 1)

accuracy_score = np.mean(y_valid == y_valid_predict)
accuracy_score

1.0

In [51]:
np.linspace(0, 8, 500).reshape(-1, 1)

array([[0.        ],
       [0.01603206],
       [0.03206413],
       [0.04809619],
       [0.06412826],
       [0.08016032],
       [0.09619238],
       [0.11222445],
       [0.12825651],
       [0.14428858],
       [0.16032064],
       [0.17635271],
       [0.19238477],
       [0.20841683],
       [0.2244489 ],
       [0.24048096],
       [0.25651303],
       [0.27254509],
       [0.28857715],
       [0.30460922],
       [0.32064128],
       [0.33667335],
       [0.35270541],
       [0.36873747],
       [0.38476954],
       [0.4008016 ],
       [0.41683367],
       [0.43286573],
       [0.4488978 ],
       [0.46492986],
       [0.48096192],
       [0.49699399],
       [0.51302605],
       [0.52905812],
       [0.54509018],
       [0.56112224],
       [0.57715431],
       [0.59318637],
       [0.60921844],
       [0.6252505 ],
       [0.64128257],
       [0.65731463],
       [0.67334669],
       [0.68937876],
       [0.70541082],
       [0.72144289],
       [0.73747495],
       [0.753